In [ ]:
%%bash
nrnivmodl

In [ ]:
# Download NEURON: http://www.neuron.yale.edu/neuron/download
# Download PyNeuronToolbox: https://github.com/ahwillia/PyNeuron-Toolbox

from neuron import h
import numpy as np
import math
import pylab as plt
import scipy.linalg
from PyNeuronToolbox.record import ez_record,ez_convert
from PyNeuronToolbox.morphology import shapeplot,shapeplot_animate
from mpl_toolkits.mplot3d import Axes3D
from JSAnimation import IPython_display
from matplotlib import animation
from matplotlib.pyplot import cm
%matplotlib inline
np.random.seed(123456789)

DC = 10 # diffusion coeff [microns^2 s^-1]

# Load morphology and other stuff
# --> SegLists: soma[2], dend[74], dend_5[37], apic[42], axon[1]
# --> Files from Migliore & Migliore (2012)
# --> CA1 pyramidal neuron
h.load_file('stdrun.hoc')
h.xopen('ri06.hoc')
h.xopen('fixnseg.hoc')
h.xopen('5a_nogui.hoc')
h.tstop = 700.0

In [ ]:
from PyNeuronToolbox.morphology import dist_between,allsec_preorder

def dist_to_soma(segment):
    return dist_between(h,h.soma[0](0.5),segment)

#seglist in pre-order
sec_list = allsec_preorder(h)
seg_list = []
for sec in sec_list:
    locs = np.linspace(0,1,sec.nseg+2)[1:-1]
    for loc in locs:
        seg_list.append(sec(loc))
N = len(seg_list)

In [ ]:
for i in range(len(sec_list)):
    sec = sec_list[i]
    print(f"{i}: {sec.name()}, nseg={sec.nseg}, L={sec.L}, or={sec.orientation()}, hname={sec.hname()}")

In [ ]:
import re
import pandas as pd
p = re.compile("^([^\\[]+)\\[.+")
tps = [p.sub("\\1",seg.sec.name()) for seg in seg_list]
pd.Series(tps).drop_duplicates().tolist()

In [ ]:
# matrix exponential used to solve linear system
from scipy.linalg import expm

# initial condition, u starts only in soma and is trafficked out to dendrites
u0 = np.zeros(4*N+1)
u0[4*N] = 1.0  # compartment 0 is the soma

In [ ]:
def trafficking_solution(utarg):
    """ Solve the problem by tuning trafficking rates, like Figs 1 and 2. """
    x = []
    
    # Iterative traversal of dendritic tree in pre-order
    i = 0
    section = None
    parentStack = [(None,h.soma[0])]
    while len(parentStack)>0:
        # Get next section to traverse
        #  --> p is parent index, section is h.Section object
        (p,section) = parentStack.pop()
        
        # Trafficking to/from parent
        if p is not None:
            mp = utarg[p] # concentration in parent
            mc = utarg[i] # concentration in child
            x.insert(0,mp/mc)
        
        # visit all segments in compartment
        for (j,seg) in enumerate(section):
            # Deal with out/into rates within compartment, just tridiag matrix
            if j>0:
                mp = utarg[i-1]
                mc = utarg[i]
                x.insert(0,mp/mc)
                
            # move onto next compartment
            i += 1
        
        # now visit children in pre-order
        child_list = list(h.SectionRef(sec=section).child)
        if child_list is not None:
            child_list.reverse()
        for c_sec in child_list:
            parentStack.append([i-1,c_sec]) # append parent index and child
    
    # return calculated guesses (flip, up/down since get_deriv pops from start)
    return np.array(x)

def animate_sim(climits=[0,0.0035]):
    u = utrace[n:,:]
    fig = plt.figure(figsize=(8,8))
    shapeax = plt.subplot(111, projection='3d')
    lines = shapeplot(h,shapeax,order='pre',lw=2)
    shapeax.view_init(elev=80,azim=-90)
    plt.title('distribution of active/detached cargo',fontweight='bold',fontsize=14)
    plt.axis('off')

    # Hack to get a colorbar
    # http://stackoverflow.com/questions/8342549/matplotlib-add-colorbar-to-a-sequence-of-line-plots
    sm = plt.cm.ScalarMappable(cmap=plt.cm.cool, norm=plt.Normalize(climits[0],climits[1])) 
    #sm._A = []
    #plt.colorbar(sm, shrink=0.5)
    #plt.colorbar(sm)
    plt.tight_layout()

    anim_func = shapeplot_animate(u.T,lines,u.shape[1],clim=climits,cmap=cm.cool)
    return animation.FuncAnimation(fig, anim_func, frames=u.shape[1], interval=5e2, blit=True)

def snap_ss(name,clim=[0,0.0035],cmap=plt.cm.cool):
    u = utrace[n:,:]
    fig = plt.figure(figsize=(8,8))
    shapeax = plt.subplot(111, projection='3d')
    lines = shapeplot(h,shapeax,cvals=u[:,-1],clim=clim,\
                      cmap=cmap,order='pre',lw=2)
    shapeax.view_init(elev=80,azim=-90)
    plt.title('steady-state',fontweight='bold',fontsize=14)

    # colorbar
    sm = cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(clim[0],clim[1])) 
    #sm = plt.cm.ScalarMappable(cmap=plt.cm.cool, norm=plt.Normalize(clim[0],clim[1])) 
    #sm._A = []
    #plt.colorbar(sm, shrink=0.5)
    #plt.colorbar(sm,cax=shapeax)
    #plt.colorbar()
    plt.tight_layout()
    plt.axis('off')
    #plt.savefig(name+'.svg')

Read Experimental Data
----

In [ ]:
tdf=pd.read_csv('data/seg_mapping.csv')
abbCA1=tdf['abb']
abbT={}
segIdx={}
for i in range(N):
    abbT[abbCA1[i]] = 1+ abbT.get(abbCA1[i],0)
    ll=segIdx.get(abbCA1[i],[])
    ll.append(i)
    segIdx[abbCA1[i]] = ll

(abbCA1,abbT,segIdx.keys(),{k:len(segIdx.get(k)) for k in segIdx.keys()})

In [ ]:
sec=seg_list[0]
sec.sec.name()

In [ ]:
sname=[]
hname=[]
for i in range(len(seg_list)):
    sec = seg_list[i].sec
    sname.append(sec.name())
    hname.append(sec.hname())
#(sname,hname)

In [ ]:
ks=[i for i in segIdx.keys()]
for i in segIdx[ks[0]]:
    sec = seg_list[i].sec
    print(f"{i}: {sec.name()}, nseg={sec.nseg}, L={sec.L}, or={sec.orientation()}, hname={sec.hname()}")

In [ ]:
expD=pd.read_csv('data/CA1_gradient.csv')
subreg = ['CA1so', 'CA1sr', 'CA1slm']
expD

In [ ]:
import sushibelt
import time

Prepare protein parameters
------

In [ ]:
lowb=np.array([0,-18,0,-18,-18,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07])
upbga=np.array([1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1])
Ndim=len(lowb)
(Ndim,len(lowb),len(upbga))

In [ ]:
bgSignal = 1e-5
cname0='D0M3'
d0w = -1 * np.ones(N)
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    d0w[sidx] *= -1*expD[f"{cname0}_MEAN"][i]/len(sidx)
for i in range(N):
    if d0w[i]<0:
        d0w[i] = bgSignal
dinit = d0w/np.sum(d0w)

cname7='D7M3'
d7w = -1*np.ones(N)
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    d7w[sidx] *= -1 * expD[f"{cname7}_MEAN"][i]/len(sidx)
for i in range(N):
    if d7w[i]<0:
        d7w[i] = bgSignal
target = np.array(expD[f"{cname7}_MEAN"])/np.sum(expD[f"{cname0}_MEAN"]) #norm target to Day0 sum to take into accound degradation
tnorm = np.sum(target ** 2)
day7 = 7 * 24 * 3600 # final time point

itarg = np.ones(N, dtype=int)
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx = segIdx[abb]
    itarg[sidx] *= (i+5)
tstart=1e10

In [ ]:
prDemand = [0.01385481065982917,-6.226265838840085,0.6709771115511489,-17.995913305482574,
            -16.696101588995024,0.21868548101204954,0.06211405206769383,0.048882353452234434,
            0.046632773486508856,1.9401449712242143e-06,0.7864443640945032,0.5632548318074286,
            0.14915702983161863,0.1491589398028194,0.13322800132469398,0.15668388533325328,
            0.17435550169553826,0.20436446923690443,0.37541096611836217,0.5260391371129066,
            0.043995168941356404,0.1970021740399707,0.19622524557390722,0.2758337024352678,0.9999999999613025]
def calcProtDemand(par,delta=bgSignal):
    F = par[0]
    Ctau = 10 ** par[1]
    mProp = par[2]
    dvA = par[3]
    dvB = par[4]
    dv = np.zeros(N)
    utarg = delta*np.ones(N)
    for k in range(N):
        if itarg[k] > 4:
            utarg[k] = par[itarg[k]]
            dv[k] = (10 ** dvA) + (10 ** dvB)*utarg[k]
    utarg /= np.sum(utarg)
    K = np.sum(utarg) / N
    x = trafficking_solution(F * utarg + (1 - F) * K)
    a = (1 / (1 + x))
    a = list(a)
    b = list((1 / (1 + x ** -1)))
    l = list(dv)
    c = list(Ctau * utarg / (F * utarg + (1 - F) * K))
    d = list(np.zeros(N))
    return a,b,c,d,l

ar,br,cr,dr,lr = calcProtDemand(prDemand)
a0=ar[0]
ar[0]=-1
ap,bp,cp,dp,lp = calcProtDemand(prDemand)
(a0,ar[0],ap[0],min(ap),max(ap),min(cp),max(cp),ar)

In [ ]:
ar_b,br_b,cr_b,dr_b,_ = calcProtDemand(prDemand)
ap_b,bp_b,cp_b,dp_b,lp_b = calcProtDemand(prDemand)

def calcUtrace(par,delta=bgSignal,tstart=1e10):
    # Soma RNA traffick rates
    ar=ar_b.copy()
    br = br_b.copy()
    ar[0] = par[0] # Soma $a_r^s$
    ar[1] = par[0]
    br[0] = par[1] # Soma $b_r^s$
    br[1] = par[1]
    # Soma protein traffick rates
    ap=ap_b.copy()
    bp = bp_b.copy()
    ap[0] = par[2] # Soma $a_p^s$
    ap[1] = par[2]
    bp[0] = par[3] # Soma $b_p^s$
    bp[1] = par[3]
    # Soma protein degradation rate
    lp = lp_b.copy()
    lp[0] = 10 ** par[4] # Soma $l_p^s$
    lp[1] = 10 ** par[4] # Soma $l_p^s$
    # Transcription rate
    tr = 10 ** par[6]
    # Soma RNA reattachment
    dr = dr_b.copy()
    dr[0] = 10 ** par[8] # Soma $d_r^s$
    dr[1] = 10 ** par[8]
    # Soma protein reattachment
    dp = dp_b.copy()
    dp[0] = 10 ** par[9] # Soma $d_p^s$
    dp[1] = 10 ** par[9]
    # Soma protein detachment
    cp = cp_b.copy()
    cp[0] = 10 ** par[11] # Soma $c_p^s$
    cp[1] = 10 ** par[11]
    # RNA detachment
    cr = np.array(cr_b) * (10 ** par[12]) # Neurophil $f_{C_{\tau}} = \frac{C_{\tau R}}{C_{\tau p}}$
    cr[0] = 10 ** par[10] # Soma $c_r^s$
    cr[1] = 10 ** par[10]
    cr = list(cr)
    # RNA degradation
    lr = np.ones(N) * (10 ** par[13]) # Neurophil $l_r^n$
    lr[0] = 10 ** par[5] # Soma $l_r^s$
    lr[1] = 10 ** par[5]
    lr = list(lr)
    # RNA translation
    tp = np.ones(N) * (10 ** par[14]) # Neurophil $t_p^n$
    tp[0] = 10 ** par[7] # Soma $t_p^s$
    tp[1] = 10 ** par[7]
    tp = list(tp)


    A = sushibelt.full_sushi_system(h, ar, br, cr, dr, tr, lr, ap, bp, cp, dp, tp, lp)
    #utrace,times = sushibelt.run_sim(h,A, npools=4,tmax=1.5,t0=tstart)
    return A #utrace.T


#par=np.array([3.5,0.5,3.5,0.5,-3,-7,-17,-20,-3,-7,-7,-7,1,-3,-7])
#par=np.array([0.5,0.5,0.5,0.5,-7,-7,-7,-7,-7,-7,-7,-7,1,-7,-7])
#par=np.array([4.0695723120091705,3.568821832094272,7.053845644866796,4.502649571353622,-9.324359618255054,-2.386911740624622,-7.017734821782558,-3.0028671244619307,-10.817587667246043,-11.666338462476672,-12.634096453636001,-8.215004440801879,-2.838167162844918,-7.5223755286280936,-18.0])
#cost=0.13059573830533733
#par=np.array([3.8228119351305088,3.877316301781682,6.61582368075308,4.220631924267806,-12.045381187498002,-2.724365509588319,-6.836501415824419,-3.1782498934472763,-13.391736840458972,-12.478317625089288,-10.860388478250565,-12.909323496429513,2.2565625387966524,-10.131398093615665,-17.97635689289944])
#ost=0.13034024969907756
#par=np.array([10,0.77708539,10,10,-7.166462208,-3.982176228,-6.374426734,-6.532612216,-9.273667623,-8.898544869,-5.892605138,-12.2193624,1.825640701,-18,-8.214616697])
#cost=0.101752713
par=np.array([1.6619156996432725e-05,10.0,50.0,4.787174316875313e-06,1.0,-10.050044051211568,1.0,-18.0,-18.0,1.0,-7.41535470425319,-18.0,2.162283411836916,-26.720975974032726,-12.563329062591443])
cost=0.10175969646368606
initTime=time.time()
A = calcUtrace(par)
#utraceR = calcUtrace(par,tstart=day7)
#utraceR = sushibelt.run_sim(h,A, npools=4, t0 = 0.3,dt = 5)
utrace,times = sushibelt.run_sim(h,A, npools=4,tmax=1.5,t0=tstart)
FinalTime=time.time()-initTime
print(FinalTime)


In [ ]:
len(par)

In [ ]:
intD=pd.read_csv('data/CA1_Day0_intensity.csv')
subreg = ['CA1so', 'CA1sr', 'CA1slm']
intD

In [ ]:
target = np.array(intD["mean_intensity"])/np.sum(intD["mean_intensity"]) #norm target to Day0 sum to take into accound degradation
tnorm = np.sum(target ** 2)
utrace=utrace.T
resM, resF = sushibelt.aggregate_segments(utrace[2*N:, -1], segIdx, expD['Abbreviation'], fun=np.sum)
cost_0=np.sum((resF - target) ** 2)/tnorm
(cost_0,tnorm,np.sum((resF - target) ** 2))

In [ ]:
{
'Soma $a_r^s$' : par[0],
'Soma $b_r^s$' : par[1],
'Soma $a_p^s$' : par[2],
'Soma $b_p^s$' : par[3],
'Soma $l_p^s$' : 10 ** par[4],
'Soma $l_r^s$' : 10 ** par[5],
'Soma $t_r$' : 10 ** par[6],
'Soma $t_p^s$' : 10 ** par[7],
'Soma $d_r^s$' : 10 ** par[8],
'Soma $d_p^s$' : 10 ** par[9],
'Soma $c_r^s$' : 10 ** par[10],
'Soma $c_p^s$' : 10 ** par[11],
'Neurophil $f_{C_{\tau}}$' : 10 ** par[12],
'Neurophil $l_r^n$' : 10 ** par[13],
'Neurophil $t_p^n$' : 10 ** par[14],
'sum_resM' : np.sum(resM),
'sum_resF' : np.sum(resF),
'sum_tot' : np.sum(resF)+np.sum(resM),
'err' : np.sum((resF - target) ** 2),
'err_norm' : np.sum((resF - target) ** 2)/tnorm,
'cost' : cost

}

In [ ]:
x=np.array(range(expD.shape[0]),dtype=int)
cname='D7W3'

plt.plot(x,target,label='Target')
#plt.plot(x,nresM,label='mobile O')
#plt.plot(x,nresF,label='detached O')
plt.plot(x,resM,label='mobile DC')
plt.plot(x,resF,label='detached DC')
plt.ylabel('Value')
plt.xlabel('Region number')
# show a legend on the plot
plt.legend()
plt.title(f"Cost = {cost}") 
# function to show the plot
#plt.show()
plt.savefig('bestFitEdita_RNA_202405101710.png')
plt.savefig('bestFitEdita_RNA_202405101710.pdf')
plt.savefig('bestFitEdita_RNA_202405101710.svg')


In [ ]:
tstart=1e10
initTime=time.time()
utraceF,timesF = sushibelt.run_sim(h,A, npools=4,tmax=1.5,t0=tstart)
FinalTime=time.time()-initTime
print(FinalTime)
utraceF=utraceF.T

In [ ]:
utracePd=pd.DataFrame(utraceF)
utracePd1=pd.DataFrame(utraceF[:N,:])
utracePd2=pd.DataFrame(utraceF[N:2*N,:])
utracePd3=pd.DataFrame(utraceF[2*N:3*N,:])
utracePd4=pd.DataFrame(utraceF[3*N:4*N,:])
(utraceF.shape,tms.shape,N,N*4,utracePd.describe(),utracePd1.describe(),utracePd2.describe(),utracePd3.describe(),utracePd4.describe())

In [ ]:
(len(utraceR),utraceR[1],day7,utraceR[1]/day7)

In [ ]:
tms=utraceR[1]
utraceR = utraceR[0].T
utracePd=pd.DataFrame(utraceR)
utracePd1=pd.DataFrame(utraceR[:N,:])
utracePd2=pd.DataFrame(utraceR[N:2*N,:])
utracePd3=pd.DataFrame(utraceR[2*N:3*N,:])
utracePd4=pd.DataFrame(utraceR[3*N:4*N,:])
(utraceR.shape,tms.shape,N,N*4,utracePd.describe(),utracePd1.describe(),utracePd2.describe(),utracePd3.describe(),utracePd4.describe())

In [ ]:
(tms,tms/day7,utraceR[:2,:],utraceR[N:N+2,:],utraceR[2*N:2*N+2,:],utraceR[3*N:3*N+2,:],utraceR[4*N:N*4+2,:])

In [ ]:
((0,[(i,A[0,i]) for i in range(A.shape[1]) if A[0,i] != 0]),
 (N,[(i,A[N,i]) for i in range(A.shape[1]) if A[N,i] != 0]),
 (2*N,[(i,A[2*N,i]) for i in range(A.shape[1]) if A[2*N,i] != 0]),
 (3*N,[(i,A[3*N,i]) for i in range(A.shape[1]) if A[3*N,i] != 0]),
(4*N,[(i,A[4*N,i]) for i in range(A.shape[1]) if A[4*N,i] != 0]))

In [ ]:
(4*N,[(i,A[i,4*N]) for i in range(A.shape[0]) if A[i,4*N] != 0])

In [ ]:
(utraceR.shape,N,N*4)
x=np.array(range(N),dtype=int)

plt.plot(x,utraceR[:N,-1],label='1')
plt.plot(x,utraceR[N:2*N,-1],label='2')
plt.plot(x,utraceR[2*N:3*N,-1],label='3')
plt.plot(x,utraceR[3*N:4*N,-1],label='4')
plt.show()

In [ ]:
resM, resF = sushibelt.aggregate_segments(utraceR[2*N:4*N, -1], segIdx, expD['Abbreviation'], fun=np.sum)
x=np.array(range(expD.shape[0]),dtype=int)

#plt.plot(x,target,label=cname)
#plt.plot(x,nresM,label='mobile O')
#plt.plot(x,nresF,label='detached O')
plt.plot(x,resM,label='mobile Protein')
plt.plot(x,resF,label='detached Protein')
plt.ylabel('Value')
plt.xlabel('Region number')
# show a legend on the plot
plt.legend()
#plt.title(f"Cost = {cost}") 
# function to show the plot
plt.show()


In [ ]:
resM, resF = sushibelt.aggregate_segments(utraceR[:2*N, -1], segIdx, expD['Abbreviation'], fun=np.sum)
x=np.array(range(expD.shape[0]),dtype=int)

#plt.plot(x,target,label=cname)
#plt.plot(x,nresM,label='mobile O')
#plt.plot(x,nresF,label='detached O')
plt.plot(x,resM,label='mobile RNA')
plt.plot(x,resF,label='detached RNA')
plt.ylabel('Value')
plt.xlabel('Region number')
# show a legend on the plot
plt.legend()
#plt.title(f"Cost = {cost}") 
# function to show the plot
plt.show()


In [ ]:
t.shape

In [ ]:
u.shape

In [ ]:
nframes=50
t = np.logspace(-0.5,8.5,nframes)
t.shape

In [ ]:
u0 = np.zeros(4*n)
u0[0] = 1.0  # compartment 0 is the soma
initTime=time.time()
utrace = [u0]
w,V = scipy.linalg.eig(A)
Vinv = np.linalg.inv(V)
t = np.logspace(-0.5,8.5,nframes)
for t_ in t: utrace.append(solve_u(w,V,Vinv,t_))

FinalTime=time.time()-initTime
print(FinalTime)


cvals=utrace[:,-1]
cvals.shape

In [ ]:
(1484/len(seg_list),len(seg_list))

In [ ]:
t.shape

In [ ]:
[d/4 for d in A.shape]

In [ ]:
(1484*2,A.shape)

In [ ]:
max(dts)

In [ ]:
from PyNeuronToolbox.morphology import shapeplot, allsec_preorder, root_indices, shapeplot_animate
roots = root_indices(allsec_preorder(h))
roots

In [ ]:
import pandas as pd
fitdt = pd.read_csv('data/Distr.csv')

In [ ]:
idx = []
for i in range(n):
    if abs(dts[i]-100)<1 :
        idx.append(1)
    else:
        idx.append(0)
sum(idx)

In [ ]:
[ (i,j) for i in range(fitdt.shape[0]) for j in range(n) if abs(dts[j]-fitdt['X'][i])<1 ]

In [ ]:
type(fitdt)

In [ ]:
seg_idx = [(i, j) for i in range(fitdt.shape[0]) for j in range(n) if abs(dts[j] - fitdt['X'][i]) < 1]
len(seg_idx)

In [ ]:
inum=3
ix=[ i for i in range(3*inum)]
(ix[:inum],ix[inum:2*inum],ix[2*inum:])

In [ ]:
[ max(u[:,r]) for r in range(u.shape[0])]

In [ ]:
range(u.shape[1])

In [ ]:
    clim=[0,0.00022]
    uplt = u[:,n:2*n]
    uplt = uplt.T
    fig = plt.figure(figsize=(8,8))
    shapeax = plt.subplot(111, projection='3d')
    lines = shapeplot(h,shapeax,cvals=uplt[:,1],clim=clim,\
                      cmap=plt.cm.cool,order='pre',lw=2)
    shapeax.view_init(elev=80,azim=-90)
    plt.title('free RNA 1',fontweight='bold',fontsize=14)

    # colorbar
    sm = cm.ScalarMappable(cmap=plt.cm.cool, norm=plt.Normalize(clim[0],clim[1])) 
    #sm = plt.cm.ScalarMappable(cmap=plt.cm.cool, norm=plt.Normalize(clim[0],clim[1])) 
    #sm._A = []
    #plt.colorbar(sm, shrink=0.5)
    #plt.colorbar(sm,cax=shapeax)
    #plt.colorbar()
    plt.tight_layout()
    plt.axis('off')


In [ ]:
    clim=[0,0.00022]
    uplt = u[:,:n]
    uplt = uplt.T
    fig = plt.figure(figsize=(8,8))
    shapeax = plt.subplot(111, projection='3d')
    lines = shapeplot(h,shapeax,cvals=uplt[:,-1],clim=clim,\
                      cmap=plt.cm.cool,order='pre',lw=2)
    shapeax.view_init(elev=80,azim=-90)
    plt.title('bounded RNA -1',fontweight='bold',fontsize=14)

    # colorbar
    sm = cm.ScalarMappable(cmap=plt.cm.cool, norm=plt.Normalize(clim[0],clim[1])) 
    #sm = plt.cm.ScalarMappable(cmap=plt.cm.cool, norm=plt.Normalize(clim[0],clim[1])) 
    #sm._A = []
    #plt.colorbar(sm, shrink=0.5)
    #plt.colorbar(sm,cax=shapeax)
    #plt.colorbar()
    plt.tight_layout()
    plt.axis('off')


In [ ]:
    clim=[0,0.00022]
    uplt = u[:,2*n:3*n]
    uplt = uplt.T
    fig = plt.figure(figsize=(8,8))
    shapeax = plt.subplot(111, projection='3d')
    lines = shapeplot(h,shapeax,cvals=uplt[:,-1],clim=clim,\
                      cmap=plt.cm.cool,order='pre',lw=2)
    shapeax.view_init(elev=80,azim=-90)
    plt.title('bound protein -1',fontweight='bold',fontsize=14)

    # colorbar
    sm = cm.ScalarMappable(cmap=plt.cm.cool, norm=plt.Normalize(clim[0],clim[1])) 
    #sm = plt.cm.ScalarMappable(cmap=plt.cm.cool, norm=plt.Normalize(clim[0],clim[1])) 
    #sm._A = []
    #plt.colorbar(sm, shrink=0.5)
    #plt.colorbar(sm,cax=shapeax)
    #plt.colorbar()
    plt.tight_layout()
    plt.axis('off')


In [ ]:
    clim=[0,0.00022]
    uplt = u[:,3*n:]
    uplt = uplt.T
    fig = plt.figure(figsize=(8,8))
    shapeax = plt.subplot(111, projection='3d')
    lines = shapeplot(h,shapeax,cvals=uplt[:,-1],clim=clim,\
                      cmap=plt.cm.cool,order='pre',lw=2)
    shapeax.view_init(elev=80,azim=-90)
    plt.title('free protein -1',fontweight='bold',fontsize=14)

    # colorbar
    sm = cm.ScalarMappable(cmap=plt.cm.cool, norm=plt.Normalize(clim[0],clim[1])) 
    #sm = plt.cm.ScalarMappable(cmap=plt.cm.cool, norm=plt.Normalize(clim[0],clim[1])) 
    #sm._A = []
    #plt.colorbar(sm, shrink=0.5)
    #plt.colorbar(sm,cax=shapeax)
    #plt.colorbar()
    plt.tight_layout()
    plt.axis('off')


In [ ]:
(ss.volume(),ss.x)

In [ ]:
seg_list[419].volume()

In [ ]:
seg_idx = [(i, j,seg_list[j].volume()) for i in range(fitdt.shape[0]) for j in range(n) if abs(dts[j] - fitdt['X'][i]) < 1]
seg_idx = np.array(seg_idx)
seg_idx

In [ ]:
u.shape

In [ ]:
resp = np.zeros((fitdt.shape[0],u.shape[0]))
resp.shape

In [ ]:
resp = np.zeros((fitdt.shape[0],u.shape[0]))
for i in range(seg_idx.shape[0]):
    ii = int(seg_idx[i,0])
    ij = int(seg_idx[i,1])
    resp[ii] += u[:,ij].T
resp

In [ ]:
dist = (resp-fitdt['V']) ** 2
dist

In [ ]:
fitdt['V']

In [ ]:
[ sum((resp[:,i] - fitdt['V']) ** 2) for i in range(resp.shape[1])]

In [ ]:
import os
?os.path.isfile